<a href="https://colab.research.google.com/github/FarrahTharwat/Sentence-Similarity/blob/main/Hugging_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [ ]:
# Sentences we want sentence embeddings for
sentences = [
    "The sun is shining",    #0
    "The weather is beautiful today",  #1
    "The sun is shining",   #2
    "It is raining cats and dogs",  #3
    "The car is red",  #4
    "The apple is red",  #5
    "This is a sentence",   #6
    "This is a completely different sentence",  #7
    "He enjoys playing football",  #8
    "Football is his favorite sport",  #9
    "The book was laid on the table",  #10
    "The table held several books",  #11
    "She loves to read horror novels",  #12
    "Reading horror stories is her hobby",  #13
    "Birds fly south in the winter",   #14
    "In winter, birds migrate south",  #15
    "He is a software engineer",   #16
    "He writes code",  #17
    "Climate change is a global issue",  #18
    "Global warming affects the earth",  #19
    "Water boils at 100 degrees Celsius",  #20
    "Boiling point of water is 100°C",  #21
    "She moved to New York last year",  #22
    "Last year, she relocated to New York",  #23
    "The museum is closed on Mondays",  #24
    "On Mondays, the museum isn't open",  #25
    "I love eating strawberries",   #26
    "Strawberries are my favorite fruit",  #27
    "They won their first soccer match",   #28
    "Their team lost the soccer game",   #29
    "He has two siblings", #30
    "The film started at nine o'clock",   #31
    "The movie began at 9 PM",  #32
    "Our cat is very old",   #33
    "Our pet is quite young",  #34
    "She's studying to become a lawyer",   #35
    "She is attending law school",  #36
    "He's allergic to peanuts",  #37
    "Peanut allergies affect him",  #38
    "They're looking forward to the trip",  #39
    "The upcoming trip excites them",  #40
    "I need to charge my phone",   #41
    "My phone battery is dead",  #42
    "Can you call me later?",   #43
     "Please phone me afterwards",  #44
    "The coffee is too hot to drink",   #45
    "Rainforests are located near the equator",   #46
    "Equatorial regions have rainforests",  #47
    "Mathematics is challenging for many students",  #48
    "Many students struggle with math",  #49
    "The Earth orbits the Sun",  #50
    "The Sun is orbited by the Earth",  #51
    "The chef cooked a delicious meal",  #52
    "A tasty dinner was prepared by the chef",  #53
    "He is reading a novel", #54
    "He is watching a movie",  #55
    "My favorite season is spring",  #56
    "I love the springtime",  #57
    "The dog barked loudly",  #58
    "A loud noise was made by the dog",  #59
    "The concert starts at eight o'clock",  #60
    "The show begins at 8 PM",  #61
    "She cut her hair short",  #62
    "She has long hair",  #63
    "They painted the room blue",  #64
    "The room was painted red",  #65
    "He runs every morning",  #66
    "Running is his morning routine",  #67
    "Lightning usually precedes thunder",  #68
    "Thunder follows lightning",  #69
    "The boy broke the window",  #70
    "The window was broken by the girl",  #71
    "It rained the whole day",  #72
    "The rain lasted all day",  #73
    "The cake recipe calls for eggs",  #74
    "Eggs are needed for the cake",  #75
    "She took a flight to Rome",  #76
    "She drove to Rome",  #77
    "Whales are mammals",   #78
    "Whales are not fish",  #79
    "The meal was very satisfying",  #80
    "Dinner was quite disappointing",  #81
    "I turned off the light",  #82
    "The light was switched off",  #83
    "He forgot his wallet at home",  #84
    "He left his wallet at home",  #85
    "The planet Mars is red",  #86
    "Mars is known as the Red Planet",  #87
    "The building was very tall",  #88
    "It was a short building",  #89
    "The painting is a masterpiece",  #90
    "The artwork is mediocre",  #91
    "She adopted a puppy",  #92
    "She adopted a kitten",  #93
    "The laptop is new",  #94
    "The computer is old",  #95
    "I learned a lot from the lecture",  #96
    "The lecture was informative",  #97
    "The phone is ringing",   #98
    "Someone is calling",  #99
    "The exam was very difficult",  #100
    "The test was easy",  #101
    "He likes to travel",  #102
    "Traveling is his hobby",  #103
    "The tree was very old",  #104
    "The ancient tree was tall"  #105
    "The drink is cold",  #106
    "He is an only child",  #107
]

In [ ]:
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [ ]:
# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

In [ ]:
# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

In [ ]:
# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

In [ ]:
# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

In [ ]:
print("Sentence embeddings:")
print(sentence_embeddings)

Sentence embeddings:
tensor([[ 0.0005,  0.1026,  0.0837,  ..., -0.0020, -0.0327,  0.0620],
        [ 0.0115,  0.1128,  0.1637,  ..., -0.0138, -0.0560,  0.0322],
        [ 0.0005,  0.1026,  0.0837,  ..., -0.0020, -0.0327,  0.0620],
        ...,
        [ 0.0307,  0.1234,  0.0148,  ..., -0.0560,  0.0121,  0.0379],
        [-0.0213,  0.0872,  0.0108,  ..., -0.0378, -0.0062,  0.0452],
        [ 0.0104,  0.0938, -0.0158,  ..., -0.0170,  0.0769,  0.0035]])


In [ ]:
from sentence_transformers import SentenceTransformer,util

In [ ]:
cos_sim=util.cos_sim(sentence_embeddings,sentence_embeddings)

In [ ]:
cos_sim

tensor([[1.0000, 0.5258, 1.0000,  ..., 0.1819, 0.2144, 0.0719],
        [0.5258, 1.0000, 0.5258,  ..., 0.1901, 0.2309, 0.0137],
        [1.0000, 0.5258, 1.0000,  ..., 0.1819, 0.2144, 0.0719],
        ...,
        [0.1819, 0.1901, 0.1819,  ..., 1.0000, 0.6112, 0.1308],
        [0.2144, 0.2309, 0.2144,  ..., 0.6112, 1.0000, 0.0602],
        [0.0719, 0.0137, 0.0719,  ..., 0.1308, 0.0602, 1.0000]])

In [ ]:

#Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_sentence_combinations.append((cos_sim[i][j], i, j))
all_sentence_combinations

[(tensor(0.5258), 0, 1),
 (tensor(1.0000), 0, 2),
 (tensor(0.2382), 0, 3),
 (tensor(0.2084), 0, 4),
 (tensor(0.2688), 0, 5),
 (tensor(0.1524), 0, 6),
 (tensor(0.0912), 0, 7),
 (tensor(0.0724), 0, 8),
 (tensor(0.0291), 0, 9),
 (tensor(0.1474), 0, 10),
 (tensor(0.0427), 0, 11),
 (tensor(0.0660), 0, 12),
 (tensor(0.0998), 0, 13),
 (tensor(0.1350), 0, 14),
 (tensor(0.1622), 0, 15),
 (tensor(0.0678), 0, 16),
 (tensor(0.0585), 0, 17),
 (tensor(0.2039), 0, 18),
 (tensor(0.2631), 0, 19),
 (tensor(0.0549), 0, 20),
 (tensor(0.0683), 0, 21),
 (tensor(0.0265), 0, 22),
 (tensor(0.0203), 0, 23),
 (tensor(0.0413), 0, 24),
 (tensor(0.1152), 0, 25),
 (tensor(0.1053), 0, 26),
 (tensor(0.1243), 0, 27),
 (tensor(0.0392), 0, 28),
 (tensor(0.0572), 0, 29),
 (tensor(0.0971), 0, 30),
 (tensor(0.1833), 0, 31),
 (tensor(0.1224), 0, 32),
 (tensor(0.0326), 0, 33),
 (tensor(0.0807), 0, 34),
 (tensor(0.0164), 0, 35),
 (tensor(0.0345), 0, 36),
 (tensor(0.0793), 0, 37),
 (tensor(-0.0146), 0, 38),
 (tensor(0.1457), 0,

In [ ]:
#Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print("Top-10 most similar pairs:")
for score, i, j in all_sentence_combinations[0:10]:
    print("{} \t {} \t {:.4f}".format(sentences[i], sentences[j], cos_sim[i][j]))

Top-10 most similar pairs:
The sun is shining 	 The sun is shining 	 1.0000
She moved to New York last year 	 Last year, she relocated to New York 	 0.9578
The Earth orbits the Sun 	 The Sun is orbited by the Earth 	 0.9360
Birds fly south in the winter 	 In winter, birds migrate south 	 0.8932
The planet Mars is red 	 Mars is known as the Red Planet 	 0.8831
Water boils at 100 degrees Celsius 	 Boiling point of water is 100°C 	 0.8790
The museum is closed on Mondays 	 On Mondays, the museum isn't open 	 0.8728
Mathematics is challenging for many students 	 Many students struggle with math 	 0.8670
He forgot his wallet at home 	 He left his wallet at home 	 0.8564
I turned off the light 	 The light was switched off 	 0.8523
